In [6]:
# 今日头条搜索框搜索后视频爬取下载——并行pool

from time import sleep
import random
import requests
import urllib.request
from multiprocessing.dummy import Pool
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
# }

def get_one_user_agent():
    with open('./user_agent_all_ok.txt', 'r', encoding='utf-8') as file:
        ret = file.readlines()
    user_agent_list = [i.strip() for i in ret]
    one_user_agent = random.choice(user_agent_list)
    return one_user_agent	

    
one_user_agent = get_one_user_agent()
print(one_user_agent)
headers={'User-Agent': one_user_agent}

option = webdriver.ChromeOptions()
# #设置无头模式
# option.add_argument("--headless")

# 关闭左上方 Chrome 正受到自动测试软件的控制的提示
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_experimental_option('useAutomationExtension', False)

#设置user-agent
# option.add_argument('user-agent=' + headers['User-Agent'])
option.add_argument('user-agent=' + one_user_agent)


# driver = webdriver.Chrome(options=option)
s = Service('./chromedriver.exe')
driver = webdriver.Chrome(service=s,options=option)
# driver = webdriver.Chrome(service=s)


driver.get('https://www.toutiao.com/')
# sleep(2)

driver.implicitly_wait(2)

# 搜索框搜索.....
search_input = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[4]/div/div[1]/input')
search_input.send_keys('乌克兰')

# 搜索按钮点击
btn = driver.find_element(By.XPATH, value='//*[@id="root"]/div/div[4]/div/div[1]/button')
btn.click()
# sleep(2)
driver.implicitly_wait(2)

# print(search_input)


Mozilla5.0 (Windows; U; Windows NT 5.1; en-US; rv1.7.5) Gecko20041107 Firefox1.0 StumbleUpon1.999


In [10]:

# 更新driver页面数据到新的页面
handle = driver.window_handles
driver.switch_to.window(handle[-1])

# 更新url到现在页面
# url = driver.current_url
# driver.get(url)
# print(url)
# sleep(2)
driver.implicitly_wait(2)

handle_main = driver.current_window_handle  # 句柄

# page_text = driver.page_source
# with open('./军事/源码.html', 'w', encoding='utf-8') as fp:
#     fp.write(page_text)
# # print(page_text)

video_btn = driver.find_element(By.XPATH, value='//*[@class="cs-view cs-view-flex align-items-center flex-row"]/a[3]')
video_btn.click()
# sleep(2)
driver.implicitly_wait(2)

video_list = driver.find_elements(By.XPATH, value='//*[@class="cs-view cs-view-block cs-grid-cell grid-cell-3 grid-cell-x-m grid-cell-y-m"]/div/div/div[1]')
# video_list = driver.find_elements(By.XPATH, value='//*[@class="cs-view cs-view-block cs-grid-cell grid-cell-3 grid-cell-x-m grid-cell-y-m"//@href')
index = 0
urls = []  # 存储所有视频的链接和文件名

# 对视频链接发起秦秋获取视频的二进制数据，然后将视频数据进行返回
def get_video_data(dic):
    url = dic['url']
    print(dic['title'],'正在下载......')
    data = requests.get(url= url,headers=headers).content
    with open(dic['title'],'wb') as fp:
        fp.write(data)
        print(dic['title'],'下载成功')

for video in video_list:
    video.click()
    a = random.random() * 3
    sleep(a)  # 设置随机休息

    # 更新driver页面数据到新的页面
    handle=driver.window_handles
    driver.switch_to.window(handle[-1])

    # 更新url到现在页面
    # video_page_url = driver.current_url
    # driver.get(video_page_url)

    page_text = driver.page_source
    tree = etree.HTML(page_text)
    src_url = 'https:' + tree.xpath('//video/@src')[0]
    print(src_url)

    title = tree.xpath('//h1/@title')[0] + '.mp4'

    if src_url.find('blob') == -1:
        # response = requests.get(url=src_url, headers=headers)
        # urllib.request.urlretrieve(src_url, './视频/' + title)
        # print(title + ".mp4视频文件下载完成！")
        
        dic = {
            'title' : title,
            'url' : src_url
        }
        urls.append(dic)
        index += 1

    if index == 3:

        print("视频链接保存完成！")
        # driver.quit()
        break

    driver.close()
    driver.switch_to.window(handle_main)

pool = Pool(4)
pool.map(get_video_data, urls)

pool.close()
pool.join()


KeyboardInterrupt: 

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}
url = 'https://v26-web.toutiaovod.com/c3c72a4c2ee224dd4b801f11cecc6944/64ecddd4/video/tos/cn/tos-cn-ve-15c001-alinc2/okCYwOg2bAQs51Xzhn9kStB8TDADeAFn2EfIVJ/?a=24&ch=0&cr=0&dr=0&er=0&lr=unwatermarked&net=5&cd=0%7C0%7C0%7C0&cv=1&br=2529&bt=2529&cs=0&ds=3&ft=Pg6SNj2-ccxiDIGusfusdex4RvMm3-Nf5s9YWHu_IAZGOjuh00v~hX8lZcS6XRf&mime_type=video_mp4&qs=0&rc=N2dpZGllNGU4NjU7OjxpNEBpanRvOmo5cmk3bTMzNDczM0A2NjJjNTQxXzExNC8xNTJeYSNfcWZmMmRrb2BgLS1kLS9zcw%3D%3D&btag=e00028000&dy_q=1693241080&l=20230829004440C07DF844AA32643115A6'
data = requests.get(url= url,headers=headers).content
with open('视频.mp4','wb') as fp:
    fp.write(data)
    print('下载成功')

下载成功
